In [1]:
!pip install qdrant-client
!pip install langchain
!pip install -U langchain-community
!pip install gradio
!pip install sentence-transformers
!pip install langchain-groq
!pip install PyPDF2
# !pip install dropbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently tak

In [ ]:
# !pip install -U langchain-huggingface

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
import time
import random
import requests
from bs4 import BeautifulSoup
from qdrant_client import QdrantClient
# from qdrant_client.http.models import VectorParams
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
# from transformers import pipeline
import gradio as gr
from google.colab import userdata

from langchain import PromptTemplate
from langchain_groq import ChatGroq
# from langchain_community.llms import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

- `chunk_size`: Typical Value: 500 to 1,000 characters
- `chunk_overlap`: Typical Value: 50 to 100 characters

In [3]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
import os
# import dropbox

In [5]:
# Secrets
qdrant_url = userdata.get('QDRANT_URL')
qdrant_api_key = userdata.get('QDRANT_API-KEY')
groq_api_key = userdata.get('GROQ_API_KEY')

# Function to extract text from PDFs

In [6]:
# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    pdf_text = ""
    with open(pdf_path, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(reader.pages)):
            pdf_text += reader.pages[page_num].extract_text()
    return pdf_text

# Function to load and extract text from different document types

In [7]:
# Function to load and extract text from different document types
def load_documents_from_directory(directory_path):
    documents = []

    # Iterate over files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Handling text files (.txt)
        if filename.endswith(".txt"):
            with open(file_path, "r") as file:
                content = file.read()
                doc = Document(page_content=content, metadata={"filename": filename})
                documents.append(doc)

        # Handling PDF files (.pdf)
        elif filename.endswith(".pdf"):
            pdf_text = extract_text_from_pdf(file_path)
            doc = Document(page_content=pdf_text, metadata={"filename": filename})
            documents.append(doc)

    return documents

In [8]:
# Step 1: Load documents from a directory (handling both .txt and .pdf)
directory_path = "/content/drive/Othercomputers/My Laptop/Training/Atomcamp/DS6_Bootcamp/Projects/FYP/Rules_and_Policies"
documents = load_documents_from_directory(directory_path)

In [9]:
len(documents)

8

In [10]:
# Step 2: Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=250)
split_docs = text_splitter.split_documents(documents)

In [ ]:
len(split_docs)

428

In [ ]:
type(split_docs)

list

In [ ]:
# split_docs
for doc in split_docs:
    print(type(doc.page_content))
    print(doc.page_content)
    print('*'*60)

<class 'str'>
KARAKORAM INTERNATIONAL UNIVERSITY  
(Gil git-Baltistan, Pakistan ) 
Directorate of Academic Planning and Review (DAPR)  
www.kiu.edu.pk ; Ph # 05811 -960010 Ext # 1 60 
 
An important meeting to review Spring semester -2023 and start of Fall semester -2023 was 
held on August 21 , 2023 at  2:00 pm in  KIU Senate Hall. All DEANS, HODs and  Directors 
attended the meeting  the worthy Vice Chancellor chaired the meeting.  
 
The follow ing decisions were made as under: - 
i) Timely Conduct of Examinations . It wa s reso lved that Ho Ds will share 
examination  sche dule  with Internal Examination Section one week prior to the 
examinations to be held.  DEANS/Associate Deans will ensure that  
examinations are conducted as per plan and results are declared in time as per 
plan.  
ii) Practical Examinations .  To be conducted one week before the final term 
examination and fed into system/LMS  before the dead line. 
iii) Closing of LMS System.  No extension/opening of LMS sha

In [ ]:
for doc in split_docs:
    # Extract the document's ID and other metadata like source
    chunk_id = doc.metadata.get('_id', 'No ID available')
    source = doc.metadata.get('source', 'No source available')

    # Print the chunk ID and source
    print(f"Chunk ID: {chunk_id}, Source: {source}")

Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: No ID available, Source: No source available
Chunk ID: 

In [ ]:
# Step 3: Embed the document chunks using HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# chunk_embeddings = embeddings.embed_documents([doc.page_content for doc in split_docs])

<ipython-input-14-5fb0bb758e14>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# # Step 3: Connect to Qdrant
# ##########################################
# # Run once!
# ##########################################
qdrant = Qdrant.from_documents(
    split_docs,
    embedding = embeddings,
    url = qdrant_url,
    prefer_grpc = True,
    api_key = qdrant_api_key,
    collection_name = "university-rules-chatbot"
)

In [ ]:
# qdrant_client = QdrantClient(
#     url = qdrant_url,
#     api_key = qdrant_api_key
#     )

In [ ]:
# Step 5: Create retriever object
retriever = qdrant.as_retriever(search_kwargs={"k": 20})

In [ ]:
# print(retriever)

In [ ]:
# Step 6: Query for relevant documents
query = "Minutes of 12th academic council meeting?"
retrieved_docs = retriever.get_relevant_documents(query)

<ipython-input-18-8cecdf264bbf>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [ ]:
# Check the retrieved documents to ensure they contain full chunks
for doc in retrieved_docs:
    print(f"Retrieved document:", doc.page_content)

    # Extract the document's ID and other metadata like source
    chunk_id = doc.metadata.get('_id', 'No ID available')
    source = doc.metadata.get('source', 'No source available')

    # Print the chunk ID and source
    print(f"\nChunk ID: {chunk_id}, Source: {source}")
    print('*' * 60)

Retrieved document: anticipation to the approval of the Academic Council. Such Undergraduate admission and semester rules/regulations KIU, 2019  (revised)  
Page 20 of 54 
 changes/corrections/amendments shall be reported to the Academic Council in its 
next/forthcoming meeting for consent.  
 
c. In case the University gets closed due to unusual circumstances, then sp ecial 
makeup classes must be arranged converting weekends or holidays to working days 
to cover the lapsed period of the students.  
d. In view of the University's Academic Calendar, chairperson/HOD of each 
Academic department (and for each discipline within the department) must publish 
an undergraduate and Graduate catalogues & pamphlets  including detailed 
schedule of complete academic activities for the year (including Spring & Fall 
semesters), A dmission requirements/Criteria,  procedure of admission (includi ng 
test/interview with a pattern question /test paper), fee structure, university financial 
aid polici

In [11]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        # Format the metadata into a string
        metadata_str = ', '.join(f"{key}: {value}" for key, value in doc.metadata.items())

        # Combine page content with its metadata
        doc_str = f"{doc.page_content}\nMetadata: {metadata_str}"

        # Append to the list of formatted documents
        formatted_docs.append(doc_str)

        # print(f"Formatted Document {len(formatted_docs)}:\n{doc_str}\n{formatted_docs}\n")  # my addition

    # Join all formatted documents with double newlines
    return "\n\n".join(formatted_docs)

## Create template

In [27]:
def retrieve_answer(question: str, bot: str):
    """
    Retrieve the answer to a question from the documents.

    Args:
        question (str): The question to answer.

    Returns:
        str: The generated answer.
    """

    prompt = PromptTemplate(
        template = """
            # Your role
            You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer
            from the scraped content to the questioner's needs from the text you are given.


            # Instructions
            Your task is to answer the question using the following pieces of retrieved context delimited by XML tags.

            <retrieved context>
            Retrieved Context:
            {context}
            </retrieved context>


            # Constraint
            1. Think deeply and multiple times about the user's question\nUser's question:\n{question}\nYou must understand the intent of their question
            and provide the most appropriate answer.
            - Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate
            response based on what you understand.
            2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
            3. Generate a concise, logical answer. When generating the answer, Do Not just list your selections, But rearrange them in context
            so that they become paragraphs with a natural flow.
            4. When you don't have retrieved context for the question or If you have a retrieved documents, but their content is irrelevant to the question,
            you should answer 'I can't find the answer to that question in the material I have'.
            5. Use five sentences maximum. Keep the answer concise but logical/natural/in-depth.
            6. At the end of the response provide metadata provided in the relevant docs,
            For example:"Metadata: page: 19, source: /content/OCR_RSCA/Analyse docs JVB + mails et convention FOOT INNOVATION.pdf'. Return Just the page and source

            Question: {question}
            Helpful Answer, formated in markdown:""",

        input_variables = ["context","question"]
    )

    embeddings_model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

    qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

    qdrant = Qdrant(
        client=qdrant_client,
        collection_name="university-rules-chatbot",
        embeddings=embeddings_model
        )

    retriever = qdrant.as_retriever(search_kwargs={"k": 20})

    # docs = retriever.get_relevant_documents(query)
    # for doc in docs:
    #   print(f"Retrieved document:", doc.page_content)
    #   print('*' * 60)

    # llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,openai_api_key=openai_api_key)

    groq_llm = ChatGroq(
        model="llama-3.1-70b-versatile",  # llma-3.1-70b-versatile
        temperature=0,
        groq_api_key=groq_api_key,
        max_retries=2,
    )

    rag_chain = (
        {"context":  retriever| format_docs, "question": RunnablePassthrough()}
        | prompt
        | groq_llm
        | StrOutputParser()
    )

    answer = rag_chain.invoke(question)

    return answer

# qdrant_url = userdata.get('QDRANT_URL')
# qdrant_api_key = userdata.get('QDRANT_API-KEY')
# groq_api_key = userdata.get('GROQ_API_KEY')

In [28]:
question = "Leave rules?"  # What are the casual leave rules?
answer = retrieve_answer(question, 'bot')
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The leave rules for University employees are as follows:

* Leave can be applied for, expressed, and sanctioned in terms of days.
* There is no limit on the grant of leave on half pay so long as it is available by conversion in the leave account.
* Leave not due may be granted on full pay, to be offset against leave to be earned in future, for a maximum period of three hundred and sixty-five days in the entire period of service.
* Special leave may be granted on full pay, when applied for a period not exceeding one hundred and thirty days, on the death of a University employee's husband.
* Leave preparatory to retirement may be taken, subject to availability, either on full pay, or partly on full pay and partly on half pay, or entirely on half pay, at the discretion of the University employee.

Metadata: filename: 1.Part II service statutes (1).pdf, _id: 25d9e53a-27b6-4fd6-9d9b-8330b577806e, _collection_name: university-rules-chatbot


In [29]:
# Create an empty list to store chatbot messages
messages = []

# Add initial instructions or welcome message
messages.append(("Hello! How can I help you today?", "KIU-bot"))

# Create Gradio chatbot with the messages list
chatbot = gr.Chatbot(value=messages)

# Create Gradio interface
gr.ChatInterface(
    fn=retrieve_answer,
    chatbot=chatbot,
    title="university-rules-chatbot",
    description="Ask any question related to Karakoram International University Gilgit-Baltistan.",
    examples=[["What courses does KIU offer?"]]
).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2556aa78b676ad3b59.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2556aa78b676ad3b59.gradio.live
